## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-25-21-50-57 +0000,nypost,Armed drones designed to neutralize school sho...,https://nypost.com/2025/08/25/us-news/armed-dr...
1,2025-08-25-21-45-00 +0000,latimes,Cartel boss 'El Mayo' pleads guilty. Will he s...,https://www.latimes.com/world-nation/story/202...
2,2025-08-25-21-43-37 +0000,nyt,"Israeli Hospital Strike Kills 20 in Gaza, Incl...",https://www.nytimes.com/2025/08/25/world/middl...
3,2025-08-25-21-39-40 +0000,nypost,"Bob Menendez’ wife, Nadine, asks for light sen...",https://nypost.com/2025/08/25/world-news/wife-...
4,2025-08-25-21-38-00 +0000,wsj,Opinion | Trump Wants to ‘Blue Slip’ the Senate,https://www.wsj.com/opinion/donald-trump-alina...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
45,trump,42
18,will,10
177,new,10
28,gaza,9
166,deal,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
110,2025-08-25-17-11-00 +0000,wsj,President Trump intensified his tough-on-crime...,https://www.wsj.com/politics/policy/trump-exec...,104
73,2025-08-25-19-21-00 +0000,nypost,"Ivanka Trump’s father-in-law, US Ambassador to...",https://nypost.com/2025/08/25/world-news/charl...,87
38,2025-08-25-20-38-55 +0000,nypost,Illinois Gov. JB Pritzker calls Trump ‘wanna b...,https://nypost.com/2025/08/25/us-news/illinois...,87
68,2025-08-25-19-40-01 +0000,nypost,Trump claims ‘good conversation’ with Putin si...,https://nypost.com/2025/08/25/us-news/trump-cl...,80
179,2025-08-25-11-00-00 +0000,nypost,Trump to sign executive order ending cashless ...,https://nypost.com/2025/08/25/us-news/presiden...,80


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
110,104,2025-08-25-17-11-00 +0000,wsj,President Trump intensified his tough-on-crime...,https://www.wsj.com/politics/policy/trump-exec...
136,62,2025-08-25-15-40-13 +0000,nypost,Keurig Dr Pepper to buy JDE Peet’s in $18B dea...,https://nypost.com/2025/08/25/business/keurig-...
78,54,2025-08-25-19-11-00 +0000,wsj,Kilmar Abrego Garcia was arrested by immigrati...,https://www.wsj.com/us-news/law/kilmar-abrego-...
101,45,2025-08-25-17-39-18 +0000,nypost,"Lil Nas X ordered into drug rehab, allowed out...",https://nypost.com/2025/08/25/us-news/lil-nas-...
73,45,2025-08-25-19-21-00 +0000,nypost,"Ivanka Trump’s father-in-law, US Ambassador to...",https://nypost.com/2025/08/25/world-news/charl...
222,34,2025-08-25-01-56-00 +0000,wsj,Here’s what you need to know about the unusual...,https://www.wsj.com/politics/policy/texas-cali...
68,33,2025-08-25-19-40-01 +0000,nypost,Trump claims ‘good conversation’ with Putin si...,https://nypost.com/2025/08/25/us-news/trump-cl...
85,33,2025-08-25-18-31-54 +0000,bbc,Five journalists among 20 killed in Israeli do...,https://www.bbc.com/news/articles/cp89rp48246o...
38,32,2025-08-25-20-38-55 +0000,nypost,Illinois Gov. JB Pritzker calls Trump ‘wanna b...,https://nypost.com/2025/08/25/us-news/illinois...
134,31,2025-08-25-15-46-00 +0000,wsj,Tech executives including VC firm Andreessen H...,https://www.wsj.com/politics/silicon-valley-la...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
